# Model Specification
We will use the Black-Litterman (BL) approach for calculating weights of assets in our portfolio. We use the classical approach with some adjustments, that will be discussed in what is to follow.

Under BL, the expected returns of the assets in our portfolio are given by
$$
\mu_{BL} = \left[ (\tau \Sigma)^{-1} + P^\top \Omega^{-1} P \right]^{-1} \left[ (\tau \Sigma)^{-1} \Pi + P^\top \Omega^{-1} Q \right],
$$
where $\mu_{BL}$ is the expected return of the portfolio, $\tau$ is the uncertainty in the prior, $\Sigma$ is the covariance matrix of asset returns, $P$ is a picking matrix, with row vectors that represent the weights used in assets for expressing relative / absolute views, $\Omega$ is the diagonal covariance matrix of confidences of our expected returns $Q$ (view specified by investor), and $\Pi$ is the implied excess return of our assets.

Now,
$$
\Pi = \lambda \Sigma w_{mkt},
$$
where $\lambda$ is the investor risk aversion co-efficient, and $w_{mkt}$ are the market capitalization weights of our assets, determined by TVL in our case (*_todo: expand on this_*).

Before we jump into modelling, let's first explore our data. We extract the largest 40 assets by TVL.

In [2]:
# Exploring the universe of data
import pandas as pd
import numpy as np
from main_app.infrastructure.defi_llama import get_pool_summary_data

# Variable to specify whether to load data from a file
load_from_file = True
save_to_file = False
file_name = "./data/all_assets_df.csv"

if load_from_file:
    # Load data from file
    df = pd.read_csv(file_name)
else:
    # Retrieve data
    pool_map = get_pool_summary_data()
    df = pd.DataFrame()
    for symbol, pools in pool_map.items():
        tvl = 0
        weighted_apy = 0
        num_pools = len(pools)
        largest_pool_id = ""
        largest_pool_tvl = 0
        for pool in pools:
            tvl += pool.tvlUsd
            weighted_apy += pool.tvlUsd * pool.apy / 100
            if pool.tvlUsd > largest_pool_tvl:
                largest_pool_tvl = pool.tvlUsd
                largest_pool_id = pool.pool

        weighted_apy = weighted_apy / tvl

        df = pd.concat([df, pd.DataFrame({'symbol': [symbol], 'tvlUsd': [tvl], 'apy': [weighted_apy], 'pools': [num_pools], 'largest_pool_id': [largest_pool_id], 'largest_pool_tvl': [largest_pool_tvl], 'largest_pool_pct_of_tvl': [largest_pool_tvl / tvl * 100]})])

    # Sort
    df = df.sort_values(by='tvlUsd', ascending=False)

    # Save the DataFrame to a CSV file
    if save_to_file:
        df.to_csv(file_name, index=False)

# Retrieve largest 40 assets by TVL
largest_assets_df = df.head(40)

# Print the first 5 rows of the largest assets DataFrame
print(largest_assets_df.head(5))

   symbol       tvlUsd       apy  pools                       largest_pool_id  \
0   STETH  23486375374  0.026812     18  747c1d2a-c668-4682-b9f9-296708a3dd90   
1   WEETH  11697415279  0.021623     55  46bd2bdf-6d92-4066-b482-e885ee172264   
2    WBTC   7084373652  0.000574    132  7e382157-b1bc-406d-b17b-facba43b716e   
3  WSTETH   6945164662  0.000693    121  e6435aae-cbe9-4d26-ab2c-a4d533db9972   
4   WBETH   6038513881  0.025876      7  80b8bf92-b953-4c20-98ea-c9653ef2bb98   

   largest_pool_tvl  largest_pool_pct_of_tvl  
0       23343461123                99.391501  
1        6250222218                53.432507  
2        4182358678                59.036393  
3        2966228157                42.709256  
4        5492332610                90.955038  


Importantly, note the _largest_pool_pct_of_tvl_, being the percentage of the total TVL (across all pools) that consists of the largest pool. We note that the largest pool is always significant enough to only look at the data from the largest pool as representative.

# Parameter estimation

## Determining the risk aversion co-efficient
We wish to determine the risk aversion coefficient, $\lambda$, that represents the amount of risk an investor is willing to take. Higher $\lambda$ means less risky, and conversely, lower $\lambda$ means higher risk tolerance.

In the Black-Litterman model used for Equities this is often set to 2.5, based on long-run estimates of equity risk premiums and volatility. Works as a good default for institutional settings.

We wish to adapt this for the defi market by considering a DeFi benchmark portfolio, to be used as an index $I$, using the same method by using:
$$
\lambda = \frac{\mathbb{E}[R_I]-r_f}{\sigma_P^2},
$$
where $R_I$ are the index (portfolio) returns, $r_f$ is the risk free rate, and $\sigma$ is the standard deviation of the index returns.

First we load the data for the largest assets that will constitute the index.

In [61]:
import os

from main_app.infrastructure.market_data import get_historical_data_for_symbol, load_symbol_to_address_mapping

symbols = largest_assets_df['symbol'].to_list()
largest_pools = largest_assets_df['largest_pool_id'].to_list()

# Get current path and join with static data path
current_path = os.getcwd()
mapping_file_path = os.path.join(current_path, "..\\static_data\\symbol_to_contract_address_map.json")
symbol_to_address_mapping = load_symbol_to_address_mapping(mapping_file_path)

# Variables to specify file operations
load_from_file = True
save_to_file = False
df_tvl_file = "./data/df_largest_assets_tvl.csv"
df_apy_file = "./data/df_largest_assets_apy.csv"
df_price_file = "./data/df_largest_assets_price.csv"

if load_from_file:
    # Load DataFrames from files
    df_tvl = pd.read_csv(df_tvl_file, index_col=0, parse_dates=True)
    df_apy = pd.read_csv(df_apy_file, index_col=0, parse_dates=True)
    df_price = pd.read_csv(df_price_file, index_col=0, parse_dates=True)
else:
    df_tvl = None
    df_apy = None
    df_price = None

    # Generate the dataframes
    for symbol, pool in zip(symbols, largest_pools):
        try:
            historic_data_df = get_historical_data_for_symbol(symbol, symbol_to_address_mapping, [pool])
        except ValueError as e:
            print(f"Value error, {e}, skipping symbol {symbol}")
            continue
        except Exception as e:
            print(f"Exception, {e}, skipping symbol {symbol}")
            continue

        print(f"{symbol} has {len(historic_data_df)} entries.")

        if 'date' in historic_data_df.columns:
            if df_tvl is None:
                # Set the DataFrame with the first symbol's data
                df_tvl = historic_data_df[['date', 'tvlUsd']].rename(columns={'tvlUsd': symbol}).set_index('date')
            else:
                # Merge without suffix, using the symbol as the column name
                df_tvl = pd.merge(df_tvl, historic_data_df[['date', 'tvlUsd']].rename(columns={'tvlUsd': symbol}).set_index('date'),
                                  on='date', how='outer')

            if df_apy is None:
                df_apy = historic_data_df[['date', 'apy']].rename(columns={'apy': symbol}).set_index('date')
            else:
                df_apy = pd.merge(df_apy, historic_data_df[['date', 'apy']].rename(columns={'apy': symbol}).set_index('date'),
                                  on='date', how='outer')

            if df_price is None:
                df_price = historic_data_df[['date', 'price']].rename(columns={'price': symbol}).set_index('date')
            else:
                df_price = pd.merge(df_price, historic_data_df[['date', 'price']].rename(columns={'price': symbol}).set_index('date'),
                                    on='date', how='outer')

    # Save DataFrames to files, if specified
    if save_to_file:
        if df_tvl is not None:
            df_tvl.to_csv(df_tvl_file)
        if df_apy is not None:
            df_apy.to_csv(df_apy_file)
        if df_price is not None:
            df_price.to_csv(df_price_file)

Next we filter data  for the index to consist of all assets that have data for the last 365 days

In [62]:
from datetime import datetime, timedelta

# Check if any of the dataframes are None at the beginning of the cell
if any(df is None for df in [df_tvl, df_apy, df_price]):
    print("Error: One or more required DataFrames (df_tvl, df_apy, df_price) are None. Cannot proceed with calculations.")
    exit()

# Set the cutoff date to a year prior to today
cutoff_date = datetime.today() - timedelta(days=365*2)

# Ensure the indexes of all DataFrames are converted to datetime
df_tvl.index = pd.to_datetime(df_tvl.index)
df_apy.index = pd.to_datetime(df_apy.index)
df_price.index = pd.to_datetime(df_price.index)

# Filter rows in df_tvl, df_apy, and df_price based on the cutoff date
df_tvl = df_tvl[df_tvl.index >= cutoff_date]
df_apy = df_apy[df_apy.index >= cutoff_date]
df_price = df_price[df_price.index >= cutoff_date]

# Remove columns with NaN values from df_tvl, df_apy, and df_price
for df in [df_tvl, df_apy, df_price]:
    # for each column (being the coin's data), if the first value is nan then remove it from the columns
    for column in df.columns:
        if pd.isna(df[column].iloc[0]):
            df.drop(column, axis=1, inplace=True)

    # now filter to ensure that all data is non nan
    df.dropna(inplace=True)

Now we calculate $\lambda$

In [76]:
# Calculate daily log returns, including the growth of our assets using the apy
df_price_log_returns = np.log(df_price / df_price.shift(1) * (1 + df_apy.shift(1) / 365.0)).dropna().to_numpy()

# Calculate weight_market and target_return
mean_market_weight_vector = df_tvl.mean().to_numpy()

# Create index and calculate prices and returns
index_0 = 100
index_weights = mean_market_weight_vector / sum(mean_market_weight_vector)
index_daily_returns = (index_weights.transpose() @ df_price_log_returns.transpose())
index_prices = index_0 * np.cumprod(np.exp(index_daily_returns))
index_prices = [index_0] + index_prices.tolist()

# Calculate index mean return and standard deviation
index_mean_return = np.mean(index_daily_returns)
index_std_dev = np.std(index_daily_returns)

# Specify risk-free rate
risk_free_rate = 0.02 / 365

# Calculate lambda
l = (index_mean_return - risk_free_rate) / index_std_dev ** 2

print(f"Lambda = {l}")

Lambda = 0.5743247315927997
